In [1]:
!pip install langchain_community neo4j langchain langchain_groq sentence-transformers langchain_huggingface langchain_nvidia_ai_endpoints


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [8]:
import langchain
print("LangChain version:", langchain.__version__)


LangChain version: 0.3.24


In [3]:

!pip install -U langchain-neo4j


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
!pip install langchain_neo4j

In [4]:
# !pip install -U langchain langchain-core langchain-community langchain-groq langsmith
import pandas as pd
import numpy as np
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain_neo4j import GraphCypherQAChain
from google.colab import userdata

# Prepare Data

In [5]:
neo4j_url = "neo4j+s://a38c58ba.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "TnAlTgGxuAsAV7UsD0FbRgoKKSmBuBSekNyGmvzJG5Y"

graph = Neo4jGraph(
    url=neo4j_url,
    username=neo4j_user,
    password=neo4j_password,
    sanitize=True,
    enhanced_schema=True
)

<ipython-input-5-620215b8bbe4>:5: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


Run this code to insert your data from csv into Neo4J

In [6]:
graph.refresh_schema()
print(graph.schema)

Node properties:
- **ResourceSource**
  - `uid`: STRING Available options: ['ResourceSource_ee3bcd00-1c94-471d-9e9e-6034315274d', 'ResourceSource_a90bf7bb-a924-4c8d-9429-c4a7dd5f33e', 'ResourceSource_c880a4ed-3b3a-4d50-ab37-c8a6388ed9b', 'ResourceSource_a131b410-737e-4a43-ae63-a48abd8c453', 'ResourceSource_a6bdf60f-12ea-4925-9955-196089fdef0', 'ResourceSource_285842d0-173c-4633-adea-f4915f1fa8a', 'ResourceSource_8551020e-aa37-460c-845a-3182b166dc4', 'ResourceSource_6b7ce365-752c-420a-9fc5-5fe71cff110', 'ResourceSource_9f62b9ae-b15a-41bc-a6ac-bb08194d87c', 'ResourceSource_f48ac10a-514f-4a09-aadf-95a61e14f6d']
  - `label`: STRING Available options: ['ResourceSource']
  - `recordValid`: BOOLEAN 
- **Item**
  - `uid`: STRING Available options: ['Item_3be8ecbc-9b0e-4b5a-b187-42d30120eea1', 'Item_dc253cee-a54f-4b53-a79b-77e601b40efb', 'Item_b6b7e72a-e557-4230-8e9c-ed162a4381d9', 'Item_dd492de4-ea25-4e5b-a655-9ac75f07b962', 'Item_a5f9ec66-b512-4a2a-9f15-b7a1ea8dd235', 'Item_87975754-75f5-4f52

In [9]:
import os
import json
from typing import List
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.graphs import Neo4jGraph


nvidia_api = "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"
llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct", nvidia_api_key = nvidia_api)

In [10]:
llm.invoke("Hi, explain me about cypher query language").content

"Cypher is a query language used for the Neo4j graph database. It's designed to allow users to query and manipulate data stored in the graph database, which is made up of nodes and relationships between them.\n\nHere's an overview of Cypher:\n\n**Key Features:**\n\n1. **Declarative syntax**: Cypher uses a declarative syntax, which means you specify what you want to retrieve or manipulate, rather than how to do it. This is similar to SQL, but with a focus on graph data structures.\n2. **Pattern matching**: Cypher uses patterns to match nodes and relationships in the graph. These patterns are used to filter and retrieve data.\n3. **Node and relationship types**: Cypher supports node and relationship types, which can be used to filter and query data.\n4. **Properties**: Nodes and relationships can have properties, which are key-value pairs that store additional data.\n5. **Indexing**: Cypher supports indexing, which allows for faster query performance.\n\n**Basic Syntax:**\n\nCypher queri

In [11]:
query_types = {
    "Simple Retrieval Queries": "These queries focus on basic data extraction, retrieving nodes or relationships based on straightforward criteria such as labels, properties, or direct relationships. Examples include fetching all nodes labeled as 'Person' or retrieving relationships of a specific type like 'EMPLOYED_BY'. Simple retrieval is essential for initial data inspections and basic reporting tasks. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Complex Retrieval Queries": "These advanced queries use the rich pattern-matching capabilities of Cypher to handle multiple node types and relationship patterns. They involve sophisticated filtering conditions and logical operations to extract nuanced insights from interconnected data points. An example could be finding all 'Person' nodes who work in a 'Department' with over 50 employees and have at least one 'REPORTS_TO' relationship. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Simple Aggregation Queries": "Simple aggregation involves calculating basic statistical metrics over properties of nodes or relationships, such as counting the number of nodes, averaging property values, or determining maximum and minimum values. These queries summarize data characteristics and support quick analytical conclusions. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Pathfinding Queries": "Specialized in exploring connections between nodes, these queries are used to find the shortest path, identify all paths up to a certain length, or explore possible routes within a network. They are essential for applications in network analysis, routing, logistics, and social network exploration. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Complex Aggregation Queries": "The most sophisticated category, these queries involve multiple aggregation functions and often group results over complex subgraphs. They calculate metrics like average number of reports per manager or total sales volume through a network, supporting strategic decision making and advanced reporting. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Verbose query": "These queries are characterized by their explicit and detailed specifications about the data retrieval process and the exact information needed. They involve elaborate instructions for navigating through complex data structures, specifying precise criteria for inclusion, exclusion, and sorting of data points. Verbose queries typically require the breakdown of each step in the querying process, from the initial identification of relevant data nodes and relationships to the intricate filtering and sorting mechanisms that must be applied. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
}

In [12]:
class Question(BaseModel):
    questions: List[str] = Field(
        description="List of relevant questions for the particular graph schema. Make sure that questions can be answered with information from the schema and that the questions are diverse as possible. Make sure that the schema and the example values contains the information that can answer the questions! Do not ask questions that cannot be answered based on the provided schema. For example, if no information about subtitles can be found in the graph, don't ask any information about subtitles. Make sure to always limit the results to less than 10 results by saying 3 users, or top 5 movies, or similar."
    )

structured_llm = llm.with_structured_output(Question)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)


system_prompt = """Your task is to generate 1000 questions that are directly related to a specific graph schema in Neo4j. Each question should target distinct aspects of the schema, such as relationships between nodes, properties of nodes, or characteristics of node types. Ensure that the questions vary in complexity, covering basic, intermediate, and advanced queries.

Avoid ambiguous questions. For clarity, an ambiguous question is one that can be interpreted in multiple ways or does not have a straightforward answer based on the schema. For example, avoid asking, "What is related to this?" without specifying the node type or relationship.
Please design each question to yield a limited number of results, specifically between 10 to 50 results. This will ensure that the queries are precise and suitable for detailed analysis and training.
The goal of these questions is to create a dataset for training AI models to convert natural language queries into Cypher queries effectively.
It is vital that the database contains information that can answer the question!
Make sure to generate 1000 questions!"""

default_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            f"{system_prompt} Follow these instructions create minimum 100 possible questions: {{instructions}}"
        ),
        HumanMessagePromptTemplate.from_template(
            "Make sure to create questions for the following graph schema:{input}\n Here are some example nodes and relationship values: {values}. Don't use any values that aren't found in the schema or in provided values. Also, do not ask questions that there is no way to answer based on the schema or provided example values. Don't include question index or the sequence of the question in the list. Make sure your question is complete and clear"
        ),
    ]
)

chain = default_prompt | structured_llm

In [13]:
graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)
schema = graph.schema
all_questions = []
for type in query_types:
  print(type)
  instructions = f"{type}: {query_types[type]}"
  values = graph.query(
            """
            MATCH (n) WHERE rand() > 0.6 WITH n LIMIT 2
            CALL { WITH n MATCH p=(n)-[*3..3]-() RETURN p LIMIT 1}
            RETURN p
            """
            )
  questions = chain.invoke( {"input": schema, "instructions": instructions, "values": values})
  all_questions.extend([{"question": el, "type": type} for el in questions.questions])

Simple Retrieval Queries
Complex Retrieval Queries


Simple Aggregation Queries


Pathfinding Queries


Complex Aggregation Queries


Verbose query


In [14]:
len(all_questions)

114

In [15]:
import random

random_elements = random.sample(all_questions, 10)

print(random_elements)

[{'question': "What are the 'description' property and 'type' property of the Type nodes?", 'type': 'Pathfinding Queries'}, {'question': 'What is a total issues reported against Oscar3 of SCR_004689 ?', 'type': 'Simple Retrieval Queries'}, {'question': 'What are the uid and the names of all Commercial Organization ?', 'type': 'Simple Retrieval Queries'}, {'question': 'What are the identifier of the AlternateIdentifier nodes in the knowledge graph whose identifier starts with "nlx_40356" and limit the answer to the first 30 items', 'type': 'Simple Aggregation Queries'}, {'question': 'What are the count of the TotalMentions nodes in the knowledge graph that are connected to a Synonym node and limit the answer to the first 30 items', 'type': 'Simple Aggregation Queries'}, {'question': 'What linked mention first linked with Brighter Ideas Inc. ?', 'type': 'Simple Retrieval Queries'}, {'question': 'What are the identifier of the Type nodes in the knowledge graph whose identifier start with 

In [16]:
all_questions_df = pd.DataFrame.from_records(all_questions)
all_questions_df.to_csv('text2cypher_questions.csv', index = False)

In [17]:
import pandas as pd

df = pd.read_csv('text2cypher_questions.csv')

print(df)


                                              question  \
0    What ResourceSource node has an Item named NuC...   
1          Which Item node has identifier SCR_004561 ?   
2    What is the proper citation of RRID with the i...   
3    Return the first 3 mentions with 'timestamp' g...   
4    What is the name of Item node 'Brighter Ideas ...   
..                                                 ...   
109  What are the Items of the ResourceSource 'Reso...   
110  What is the label name of the edge between 'It...   
111  What are the 5 ResourceSources from an Authori...   
112  Brighter Ideas Inc.. What are its ResourceSour...   
113  What Items are there that are an RRID Item 'It...   

                         type  
0    Simple Retrieval Queries  
1    Simple Retrieval Queries  
2    Simple Retrieval Queries  
3    Simple Retrieval Queries  
4    Simple Retrieval Queries  
..                        ...  
109             Verbose query  
110             Verbose query  
111            

In [18]:
from IPython.display import display, HTML

# Optional: Reset and rename index if needed
df.reset_index(inplace=True)
df.index += 1  # Start from 1 instead of 0
df.rename(columns={'index': 'original_index'}, inplace=True)

# Hide the dataframe index in display
display(HTML(df.to_html(index=False)))


original_index,question,type
0,What ResourceSource node has an Item named NuChart ?,Simple Retrieval Queries
1,Which Item node has identifier SCR_004561 ?,Simple Retrieval Queries
2,What is the proper citation of RRID with the identifier RRID:SCR_004561 ?,Simple Retrieval Queries
3,Return the first 3 mentions with 'timestamp' greater than '2025-02-14 18:44:44' ?,Simple Retrieval Queries
4,What is the name of Item node 'Brighter Ideas Inc.' ?,Simple Retrieval Queries
5,What is the parental organization of ResourceSource with uid ResourceSource_ee3bcd00-1c94-471d-9e9e-6034315274d7 ?,Simple Retrieval Queries
6,Which disease is related to Item node Oscar3 ?,Simple Retrieval Queries
7,Which organization in 'HIERARCHY' is 'Parent' of organization with the name 'Brighter Ideas Inc.' ?,Simple Retrieval Queries
8,What are the uid and the names of all Commercial Organization ?,Simple Retrieval Queries
9,What are the issues associated with ResourceSource with name TETRA ?,Simple Retrieval Queries


## Leverage LLM to generate the cypher query for each generated questions

In [19]:
class CypherQuery(BaseModel):
    cypherquery: str = Field(
        description="A correct Neo4J Cypher Query Language without any preambles. Make sure that it follows the schema"
    )


structured_llm_cypher_query = llm.with_structured_output(CypherQuery)

system_prompt_cypher_query = """Given an input question, convert it to a Cypher query. No pre-amble.
Additional instructions:
- Ensure that queries checking for non-null properties use `IS NOT NULL` in a straightforward manner.
- Don't use `size((n)--(m))` for counting relationships. Instead use the new `count{{(n)--(m))}}` syntax.
- Incorporate the new existential subqueries in examples where the query needs to check for the existence of a pattern.
- give only one and only one query
  Example: MATCH (p:Person)-[r:IS_FRIENDS_WITH]->(friend:Person)
            WHERE exists{{ (p)-[:WORKS_FOR]->(:Company {{name: 'Neo4j'}})}}
            RETURN p, r, friend"""

default_prompt_cypher_query = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            f"{system_prompt_cypher_query}"
        ),
        HumanMessagePromptTemplate.from_template(
            """Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question: {schema}"
              Question: {question}
              Cypher query:"""),
    ]
)

chain_cypher_query = default_prompt_cypher_query | structured_llm_cypher_query

In [20]:
chain_cypher_query.invoke({"schema": graph.schema, "question":"Can you list all the items associated with the RRID RRID:SCR_004561?"}).cypherquery

"MATCH (rrid:RRID {curie: 'RRID:SCR_004561'})<-[r:HAS_RRID]-(item:Item) RETURN item.uid, item.name"

In [31]:
import time
import random
import pandas as pd
from langchain_community.graphs import Neo4jGraph  # adjust import if needed

# Connect to Neo4j and get schema
graph = Neo4jGraph(url=neo4j_url, username=neo4j_user, password=neo4j_password, sanitize=True, enhanced_schema=True)
schema = graph.schema

# Generate Cypher query using LLM
def generate_query(question):
    return chain_cypher_query.invoke({"schema": schema, "question": question}).cypherquery

# Try executing the query
def execute_query(query):
    return graph.query(query)

# Main logic
def get_cypher_query_with_feedback(question):
    # First attempt
    query = generate_query(question)
    try:
        execute_query(query)
        return query  # Success
    except Exception as e:
        error_msg = str(e)
        print(f"Initial query failed: {error_msg}")

        # Retry with feedback
        feedback_prompt = (
            f"Your last Cypher query failed with this error:\n"
            f"{error_msg}\n"
            f"Please correct the query for the question: {question}"
        )
        corrected_query = generate_query(feedback_prompt)

        try:
            execute_query(corrected_query)
            print("Fixed query worked!")
        except Exception as e2:
            print(f"Retry also failed: {e2}")

        # Return corrected query anyway
        return corrected_query

# Retry wrapper
def safe_get_cypher_query(question):
    retry_count = 3
    for attempt in range(retry_count):
        try:
            result = get_cypher_query_with_feedback(question)
            print(question, ':', result)
            return result
        except Exception as e:
            print(f"Error: {e}. Retrying ({attempt + 1}/{retry_count})...")
            time.sleep(random.uniform(1, 3))
    return None


# Apply to your DataFrame
df['cypher'] = df['question'].apply(lambda q: safe_get_cypher_query(q))


What ResourceSource node has an Item named NuChart ? : MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item) WHERE i.name='NuChart' RETURN r.uid
Which Item node has identifier SCR_004561 ? : MATCH (item:Item {identifier: 'SCR_004561'}) RETURN item
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (1/3)...
What is the proper citation of RRID with the identifier RRID:SCR_004561 ? : MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) RETURN rrid.properCitation
Return the first 3 mentions with 'timestamp' greater than '2025-02-14 18:44:44' ? : MATCH (n:Mentions) WHERE n.timestamp > '2025-02-14 18:44:44' RETURN n.uid, n.timestamp LIMIT 3
What is the name of Item node 'Brighter Ideas Inc.' ? : MATCH (i:Item {name: 'Brighter Ideas Inc.'}) RETURN i.name


What is the parental organization of ResourceSource with uid ResourceSource_ee3bcd00-1c94-471d-9e9e-6034315274d7 ? : MATCH (s:Supercategory {name: 'Resource'})<-[sv:HAS_SUPERCATEGORY]-(i:Item)-[:IS_RELATED_TO*]->(rr:Relationship {name: 'is_parent_organization_of'})-[:IS_EQUIVALENT_TO*]->(rr2:Relationship {name: 'is_parent_organization_of'})-[:RELATES*]->(o:Organization)-[:IS_RELATED_TO*1..2]->(rs:ResourceSource {uid: 'ResourceSource_ee3bcd00-1c94-471d-9e9e-6034315274d7'}) RETURN o.name


Which disease is related to Item node Oscar3 ? : MATCH (d:Diseases)-[:HAS_RELATED]-(s:Disease)-[:IS_RELATED_TO]-(i:Item {name: 'Oscar3'}) RETURN DISTINCT d.uid, s.uid, s.name


Which organization in 'HIERARCHY' is 'Parent' of organization with the name 'Brighter Ideas Inc.' ? : MATCH (child:Organization {name: 'Brighter Ideas Inc.'}) - [:HAS_HIERARCHY] -> (parent:HIERARCHY) RETURN child, parent
What are the uid and the names of all Commercial Organization ? : MATCH (item:Supercategory {name: 'Commercial Organization'}) RETURN item.uid, item.name
What are the issues associated with ResourceSource with name TETRA ? : MATCH (n:ResourceSource {name: 'TETRA'})-[]-(m:Issues) RETURN m
Return first 10 matching ResourceSource node with label 'ResourceSource' ? : MATCH (resource:ResourceSource {label: 'ResourceSource'}) RETURN resource.uid LIMIT 10


Return all provenance which has primary source having keyword this resource is no longer in service ? : MATCH (a:ResourceSource)-[:HAS_PROVENANCE]->(p:Provenance)<-[:HAS_PRIMARY_SOURCE]-(r:ResourceSource)-[:HAS_AVAILABILITY]->(av:Availability) WHERE av.keyword="THIS RESOURCE IS NO LONGER IN SERVICE" RETURN p
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (1/3)...


...

What is the 'SCR identifier of the ResourceSource 'ResourceSource_a131b410-737e-4a43-ae63-a48abd8c453'? : MATCH (p:Item)-[:IS_PART_OF]->(r:ResourceSource {uid: 'ResourceSource_a131b410-737e-4a43-ae63-a48abd8c453'}) RETURN p.curie
What are the labels of the adjacency edges among Item 'Item_dc253cee-a54f-4b53-a79b-77e601b40efb'? : MATCH (g:Graph)-[:HAS_RELATIONSHIPS]->(r)<-[:HAS_CHILD]-(p:ParentRelationship {identifier: 'Item_dc253cee-a54f-4b53-a79b-77e601b40efb'}) RETURN r.uid
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (1/3)...
Initial query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'Pearsonf': expected a parameter, '&', '*', ':', 'WHERE', ']', '{' or '|' (line 1, column 115 (offset: 114))
"MATCH (child:Item {uid: 'Item_dd492de4-ea25-4e5b-a655-9ac75f07b962'})-[:IS_LISTED_BY|IS_RELATED_TO|IS_CHILD_OF|IS Pearsonf]->(parent) RETURN collect(distinct parent.label) AS LabelsOfParentNodes"
                         

What are the labels of the parent nodes of 'Item_dd492de4-ea25-4e5b-a655-9ac75f07b962'? : MATCH (child:Item {uid: 'Item_dd492de4-ea25-4e5b-a655-9ac75f07b962'})<-[rel:PARENT_OF]-(parent) RETURN parent.label
What is the label name of the edge between 'Item_bfee9588-2e1b-4075-9ebd-9bd2754fe4d0' and 'ResourceSource_6b7ce365-752c-420a-9fc5-5fe71cff110'? : MATCH (:`Item` {`uid`: 'Item_bfee9588-2e1b-4075-9ebd-9bd2754fe4d0'})-[r]-(:`ResourceSource` {`uid`: 'ResourceSource_6b7ce365-752c-420a-9fc5-5fe71cff110'}) RETURN type(r), r
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (1/3)...
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (2/3)...
Brighter Ideas Inc.. What Item is it found in? : MATCH (r:RRID {name: 'Brighter Ideas Inc.'})<-[:HAS_RRID]-(i:Item) RETURN i.uid, i.name


NUI Galway. What is the ResourceSource of that Abbreviation node? : MATCH (n:ResourceSource)-[:HAS_ITEM]->(:Item)<-[:IS_ABBREVIATION_OF]-(a:Abbreviation {name: 'NUI Galway'}) RETURN n
What Items does ResourceSource 'ResourceSource_ee3bcd00-1c94-471d-9e9e-6034315274d7' have? : MATCH (i:ResourceSource {uid: 'ResourceSource_ee3bcd00-1c94-471d-9e9e-6034315274d7'})-[:HAS_ITEM]->(item:Item) RETURN item.uid
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (1/3)...
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (2/3)...


What is the type of the ResourceSource 'ResourceSource_f48ac10a-514f-4a09-aadf-95a61e14f6d' with primary key 'SCR_004689' that is labeled 'ResourceSource'? : MATCH (ResourceSource)-[:HAS_PROVENANCE]->(Provenance:Provenance{primaryKey:'SCR_004689'}), (ResourceSource)-[:HAS_TYPE]->(Type:Type) WHERE ResourceSource.uid = 'ResourceSource_f48ac10a-514f-4a09-aadf-95a61e14f6d' RETURN Type
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (1/3)...
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (2/3)...
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (3/3)...
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (1/3)...
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (2/3)...
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (3/3)...
What is the label name of the edge betwe

nan : MATCH (parent:ResourceSource)-[:HAS_child]->(child) RETURN parent,child


In [22]:
# import time

# graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)
# schema = graph.schema

# def get_cypher_query(question):
#     cypher_query = chain_cypher_query.invoke({"schema": schema, "question": question}).cypherquery
#     time.sleep(1)
#     return cypher_query

# def safe_get_cypher_query(question):
#     retry_count = 3
#     for attempt in range(retry_count):
#         try:
#             result = get_cypher_query(question)
#             print(question, ':', result)
#             return result
#         except Exception as e:
#             print(f"Error: {e}. Retrying ({attempt + 1}/{retry_count})...")
#             time.sleep(random.uniform(1, 3))
#     return None

# df['cypher'] = df['question'].apply(lambda q: safe_get_cypher_query(q))

What ResourceSource node has an Item named NuChart ? : MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item) WHERE i.name='NuChart' RETURN r.uid
Which Item node has identifier SCR_004561 ? : MATCH (item:Item {identifier: 'SCR_004561'}) RETURN item
What is the proper citation of RRID with the identifier RRID:SCR_004561 ? : MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) RETURN rrid.properCitation
Return the first 3 mentions with 'timestamp' greater than '2025-02-14 18:44:44' ? : MATCH (n:Mentions) WHERE n.timestamp > '2025-02-14 18:44:44' RETURN n.uid, n.label, n.timestamp LIMIT 3
What is the name of Item node 'Brighter Ideas Inc.' ? : MATCH (i:Item) WHERE exists{ (i:Item)-[:HAS_RESOURCE]-> (:ResourceSource {name: 'Brighter Ideas Inc.'})} RETURN i.name
Error: [429] Too Many Requests
{'status': 429, 'title': 'Too Many Requests'}. Retrying (1/3)...
What is the parental organization of ResourceSource with uid ResourceSource_ee3bcd00-1c94-471d-9e9e-6034315274d7 ? : MATCH (s:Supercategory {name: 'Reso

In [33]:
df.to_csv('raw_text2cypher.csv', index = False)

In [34]:
# Optional: Reset index if needed
df.reset_index(inplace=True)
df.index += 1  # Start index from 1 instead of 0
df.rename(columns={'index': 'original_index'}, inplace=True)

# Now display it
from IPython.display import display
display(df)

,original_index,original_index,original_index,question,type,cypher,syntax_error,timeout,returns_results
1,1,1.0,0.0,What ResourceSource node has an Item named NuC...,Simple Retrieval Queries,MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item)...,False,False,True
2,2,2.0,1.0,Which Item node has identifier SCR_004561 ?,Simple Retrieval Queries,MATCH (item:Item {identifier: 'SCR_004561'}) R...,False,False,True
3,3,3.0,2.0,What is the proper citation of RRID with the i...,Simple Retrieval Queries,MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) R...,False,False,True
4,4,4.0,3.0,Return the first 3 mentions with 'timestamp' g...,Simple Retrieval Queries,MATCH (n:Mentions) WHERE n.timestamp > '2025-0...,False,False,False
5,5,5.0,4.0,What is the name of Item node 'Brighter Ideas ...,Simple Retrieval Queries,MATCH (i:Item {name: 'Brighter Ideas Inc.'}) R...,False,False,False
...,...,...,...,...,...,...,...,...,...
111,111,111.0,110.0,What is the label name of the edge between 'It...,Verbose query,MATCH (item1:Item {uid: 'Item_dc253cee-a54f-4b...,False,False,False
112,112,112.0,111.0,What are the 5 ResourceSources from an Authori...,Verbose query,MATCH (a:Authority)<-[:HAS_AUTHORITY]-(r:Resou...,False,False,True
113,113,113.0,112.0,Brighter Ideas Inc.. What are its ResourceSour...,Verbose query,MATCH (p:ResourceSource)-[r]->(x) where p.name...,False,False,True
114,114,114.0,113.0,What Items are there that are an RRID Item 'It...,Verbose query,MATCH (n:ResourceSource {uid: 'ResourceSource_...,False,False,False


In [ ]:
df = df.drop(index=81)


In [35]:
display(df)

,original_index,original_index,original_index,question,type,cypher,syntax_error,timeout,returns_results
1,1,1.0,0.0,What ResourceSource node has an Item named NuC...,Simple Retrieval Queries,MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item)...,False,False,True
2,2,2.0,1.0,Which Item node has identifier SCR_004561 ?,Simple Retrieval Queries,MATCH (item:Item {identifier: 'SCR_004561'}) R...,False,False,True
3,3,3.0,2.0,What is the proper citation of RRID with the i...,Simple Retrieval Queries,MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) R...,False,False,True
4,4,4.0,3.0,Return the first 3 mentions with 'timestamp' g...,Simple Retrieval Queries,MATCH (n:Mentions) WHERE n.timestamp > '2025-0...,False,False,False
5,5,5.0,4.0,What is the name of Item node 'Brighter Ideas ...,Simple Retrieval Queries,MATCH (i:Item {name: 'Brighter Ideas Inc.'}) R...,False,False,False
...,...,...,...,...,...,...,...,...,...
111,111,111.0,110.0,What is the label name of the edge between 'It...,Verbose query,MATCH (item1:Item {uid: 'Item_dc253cee-a54f-4b...,False,False,False
112,112,112.0,111.0,What are the 5 ResourceSources from an Authori...,Verbose query,MATCH (a:Authority)<-[:HAS_AUTHORITY]-(r:Resou...,False,False,True
113,113,113.0,112.0,Brighter Ideas Inc.. What are its ResourceSour...,Verbose query,MATCH (p:ResourceSource)-[r]->(x) where p.name...,False,False,True
114,114,114.0,113.0,What Items are there that are an RRID Item 'It...,Verbose query,MATCH (n:ResourceSource {uid: 'ResourceSource_...,False,False,False


## Evaluate the generated Cypher query by running it against the database. Check if it has any syntax errors, times out, or returns a result.

In [26]:
syntax_error = []
returns_results = []
timeouts = []

graph = Neo4jGraph(url=neo4j_url, username=neo4j_user, password=neo4j_password, sanitize=True, enhanced_schema=True)

for i, row in df.reset_index().iterrows():
    try:
        data = graph.query(row["cypher"])
        returns_results.append(bool(data))
        syntax_error.append(False)
        timeouts.append(False)
    except ValueError as e:
        if hasattr(e, "code") and e.code == "Neo.ClientError.Statement.SyntaxError":
            syntax_error.append(True)
            print(f"Syntax error in Cypher query: {e}")
        else:
            syntax_error.append(False)
            print(f"Other ValueError: {e}")
        returns_results.append(False)
        timeouts.append(False)
    except Exception as e:
        if hasattr(e, "code") and e.code == "Neo.ClientError.Transaction.TransactionTimedOutClientConfiguration":
            timeouts.append(True)
        else:
            print(f"Other Exception: {e}")
            timeouts.append(False)
        syntax_error.append(False)
        returns_results.append(False)

Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'Relationship': expected a parameter, ')', ':', 'IS', 'WHERE' or '{' (line 1, column 142 (offset: 141))
"MATCH (rs:ResourceSource {uid: 'ResourceSource_a131b410-737e-4a43-ae63-a48abd8c453'})-[:HAS_RELATIONSHIP {type: 'parentOrganizationOf'}]->(r Relationship)-[:HAS_RESOURCE]->(parent:Organization) RETURN parent.name"
                                                                                                                                              ^}
Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'S

Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"None"
 ^}


Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '': expected an expression (line 1, column 56 (offset: 55))
"MATCH (abbr: Abbreviation) WHERE abbr.name STARTS WITH"
                                                        ^}


Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"None"
 ^}
Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'S

Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: A pattern expression should only be used in order to test the existence of a pattern. It should therefore only be used in contexts that evaluate to a boolean, e.g. inside the function exists() or in a WHERE-clause. No other uses are allowed, instead they should be replaced by a pattern comprehension. (line 1, column 55 (offset: 54))
"MATCH (sc:Supercategory) RETURN sc.uid, sc.name, SIZE((sc)<-[:HAS_SUPERCATEGORY]-()) AS item_count LIMIT 3"
                                                       ^}
Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: A pattern expression should only be used in order to test the existence of a pattern. It can no longer be used inside the function size(), an alternative is to replace size() with COUNT {}. (line 1, column 96 (offset: 95))
"MATCH (n:Item)-[:HAS_SYNONYMS]->(:Synonym)<-[:HAS_SYNONYMS]-(item:Item) RETURN n ORDER BY size((n)-[:HAS_SYNONYMS]->(:Synonym)) 

Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"None"
 ^}
Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'S

Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'TO': expected a parameter, '&', '*', ':', 'WHERE', ']', '{' or '|' (line 1, column 84 (offset: 83))
"MATCH (child:Item {uid: 'Item_dd492de4-ea25-4e5b-a655-9ac75f07b962'})-[:IS_related TO]->(parent) RETURN parent.label"
                                                                                    ^}
Other Exception: {code: Neo.ClientError.Statement.SyntaxError} {message: Type mismatch: expected List<T> but was Path (line 1, column 134 (offset: 133))
"MATCH p=(n {uid: 'Item_dc253cee-a54f-4b53-a79b-77e601b40efb'})-[]-(m {uid: 'Item_3be8ecbc-9b0e-4b5a-b187-42d30120eea1'}) RETURN type(p[0])"
                                                                                                                                      ^}


In [32]:
# len(returns_results)
len(syntax_error)

114

In [ ]:
# len(syntax_error)
df["syntax_error"] = syntax_error
df["timeout"] = timeouts
df["returns_results"] = returns_results

In [ ]:
final_df = df[["question","cypher","type","syntax_error","timeout","returns_results",]]
final_df.to_csv('detailed_text2cypher.csv', index = False)
final_df.head()


,question,cypher,type,syntax_error,timeout,returns_results
1,What is the RRID proper citation of an Item as...,MATCH (r:Item {name: 'Brighter Ideas Inc.'})--...,Simple Retrieval Queries,False,False,False
2,Can you list the Synonym names of all Items wi...,MATCH (a:Synonym)<-[:HAS_SYNONYMS]-(i:Item)<-[...,Simple Retrieval Queries,False,False,False
3,Can you list the names of all Supercategories ...,MATCH (supercategories:Supercategory)-[:CATALO...,Simple Retrieval Queries,False,False,False
4,What are the Authority names of all ResourceSo...,MATCH (resource:ResourceSource)-[:HAS_AUTHORIT...,Simple Retrieval Queries,False,False,False
5,Can you list the count values of TotalMentions...,MATCH (t:TotalMentions)<-[:HAS_MENTIONS]-(r:Re...,Simple Retrieval Queries,False,False,False


In [38]:
display(df)

,original_index,original_index,original_index,question,type,cypher,syntax_error,timeout,returns_results
1,1.0,1.0,0.0,What ResourceSource node has an Item named NuC...,Simple Retrieval Queries,MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item)...,False,False,True
2,2.0,2.0,1.0,Which Item node has identifier SCR_004561 ?,Simple Retrieval Queries,MATCH (item:Item {identifier: 'SCR_004561'}) R...,False,False,True
3,3.0,3.0,2.0,What is the proper citation of RRID with the i...,Simple Retrieval Queries,MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) R...,False,False,True
4,4.0,4.0,3.0,Return the first 3 mentions with 'timestamp' g...,Simple Retrieval Queries,MATCH (n:Mentions) WHERE n.timestamp > '2025-0...,False,False,True
5,5.0,5.0,4.0,What is the name of Item node 'Brighter Ideas ...,Simple Retrieval Queries,MATCH (i:Item {name: 'Brighter Ideas Inc.'}) R...,False,False,False
...,...,...,...,...,...,...,...,...,...
112,112.0,112.0,111.0,What are the 5 ResourceSources from an Authori...,Verbose query,MATCH (a:Authority)<-[:HAS_AUTHORITY]-(r:Resou...,False,False,False
113,113.0,113.0,112.0,Brighter Ideas Inc.. What are its ResourceSour...,Verbose query,MATCH (p:ResourceSource)-[r]->(x) where p.name...,False,False,True
114,114.0,114.0,113.0,What Items are there that are an RRID Item 'It...,Verbose query,MATCH (n:ResourceSource {uid: 'ResourceSource_...,False,False,False
115,0.0,NaN,NaN,NaN,NaN,MATCH (parent:ResourceSource)-[:HAS_child]->(c...,True,False,False


In [37]:

from neo4j.exceptions import Neo4jError, ClientError, CypherSyntaxError

df["syntax_error"] = False
df["timeout"] = False
df["returns_results"] = False
count = 0;


for i, row in df.reset_index().iterrows():
    # count += 1
    try:
        data = graph.query(row["cypher"])
        df.at[i, "returns_results"] = bool(data)

    except Neo4jError as e:
        if "SyntaxError" in str(e):
            df.at[i, "syntax_error"] = True
            print(f"Syntax error in row {i}: {str(e)[:200]}...")
        elif "InvalidInputError" in str(e):
            df.at[i, "syntax_error"] = True
            print(f"Invalid input in row {i}: {str(e)[:200]}...")
        else:
            print(f"Unexpected Neo4j error in row {i}: {str(e)[:200]}...")

    except Exception as e:
        if hasattr(e, 'code') and e.code == "Neo.ClientError.Transaction.TransactionTimedOutClientConfiguration":
            df.at[i, "timeout"] = True
            print(f"Timeout in row {i}")
        else:
            print(f"Unexpected error in row {i}: {str(e)[:200]}...")

# Verification
print(f"Processed {len(df)} queries")
print(f"- Syntax errors: {df['syntax_error'].sum()}")
print(f"- Timeouts: {df['timeout'].sum()}")
print(f"- Queries with results: {df['returns_results'].sum()}")

Syntax error in row 13: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...


Syntax error in row 28: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...
Syntax error in row 29: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...


Syntax error in row 40: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...


Syntax error in row 62: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...


Syntax error in row 64: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...
Syntax error in row 67: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...
Syntax error in row 68: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...


Syntax error in row 74: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...


Syntax error in row 83: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...
Syntax error in row 91: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...
Syntax error in row 93: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...
Syntax error in row 94: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...


Syntax error in row 108: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...
Syntax error in row 109: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'None': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DA...


Syntax error in row 115: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'nan': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DAT...
Processed 116 queries
- Syntax errors: 16
- Timeouts: 0
- Queries with results: 44


In [39]:
display(df)

,original_index,original_index,original_index,question,type,cypher,syntax_error,timeout,returns_results
1,1.0,1.0,0.0,What ResourceSource node has an Item named NuC...,Simple Retrieval Queries,MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item)...,False,False,True
2,2.0,2.0,1.0,Which Item node has identifier SCR_004561 ?,Simple Retrieval Queries,MATCH (item:Item {identifier: 'SCR_004561'}) R...,False,False,True
3,3.0,3.0,2.0,What is the proper citation of RRID with the i...,Simple Retrieval Queries,MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) R...,False,False,True
4,4.0,4.0,3.0,Return the first 3 mentions with 'timestamp' g...,Simple Retrieval Queries,MATCH (n:Mentions) WHERE n.timestamp > '2025-0...,False,False,True
5,5.0,5.0,4.0,What is the name of Item node 'Brighter Ideas ...,Simple Retrieval Queries,MATCH (i:Item {name: 'Brighter Ideas Inc.'}) R...,False,False,False
...,...,...,...,...,...,...,...,...,...
112,112.0,112.0,111.0,What are the 5 ResourceSources from an Authori...,Verbose query,MATCH (a:Authority)<-[:HAS_AUTHORITY]-(r:Resou...,False,False,False
113,113.0,113.0,112.0,Brighter Ideas Inc.. What are its ResourceSour...,Verbose query,MATCH (p:ResourceSource)-[r]->(x) where p.name...,False,False,True
114,114.0,114.0,113.0,What Items are there that are an RRID Item 'It...,Verbose query,MATCH (n:ResourceSource {uid: 'ResourceSource_...,False,False,False
115,0.0,NaN,NaN,NaN,NaN,MATCH (parent:ResourceSource)-[:HAS_child]->(c...,True,False,False


In [40]:
num_syntax_errors = df['syntax_error'].sum()
num_times_out = df['timeout'].sum()
num_returns_results = df['returns_results'].sum()

print(f"Number of syntax errors: {num_syntax_errors}")
print(f"Number of timeout: {num_times_out}")
print(f"Number of return result: {num_returns_results}")

Number of syntax errors: 16
Number of timeout: 0
Number of return result: 44


In [41]:
df_filtered = df[df["returns_results"] == True].copy()


In [42]:
display(df_filtered)

,original_index,original_index,original_index,question,type,cypher,syntax_error,timeout,returns_results
1,1.0,1.0,0.0,What ResourceSource node has an Item named NuC...,Simple Retrieval Queries,MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item)...,False,False,True
2,2.0,2.0,1.0,Which Item node has identifier SCR_004561 ?,Simple Retrieval Queries,MATCH (item:Item {identifier: 'SCR_004561'}) R...,False,False,True
3,3.0,3.0,2.0,What is the proper citation of RRID with the i...,Simple Retrieval Queries,MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) R...,False,False,True
4,4.0,4.0,3.0,Return the first 3 mentions with 'timestamp' g...,Simple Retrieval Queries,MATCH (n:Mentions) WHERE n.timestamp > '2025-0...,False,False,True
8,8.0,8.0,7.0,Which organization in 'HIERARCHY' is 'Parent' ...,Simple Retrieval Queries,MATCH (child:Organization {name: 'Brighter Ide...,False,False,True
10,10.0,10.0,9.0,What are the issues associated with ResourceSo...,Simple Retrieval Queries,MATCH (n:ResourceSource {name: 'TETRA'})-[]-(m...,False,False,True
12,12.0,12.0,11.0,Return all provenance which has primary source...,Simple Retrieval Queries,MATCH (a:ResourceSource)-[:HAS_PROVENANCE]->(p...,False,False,True
17,17.0,17.0,16.0,What are the SCORE of RRID with identifier SCR...,Simple Retrieval Queries,MATCH (item:RRID {curie: 'SCR_004689'}) RETURN...,False,False,True
18,18.0,18.0,17.0,Return top 3 Item node with identifier SCR_004...,Simple Retrieval Queries,MATCH (n:Item {identifier: 'SCR_004561'}) RETU...,False,False,True
24,24.0,24.0,23.0,Can you list the keywords of all Item nodes in...,Complex Retrieval Queries,MATCH (o:Organism)<-[:IS_RELATED_TO]-(r:Resour...,False,False,True


In [43]:
df_filtered.to_csv("filtered_results.csv", index=False)


In [44]:
df.to_csv("results.csv", index=False)
